# Selenium (https://www.selenium.dev)
- 웹 브라우저를 자동으로 제어할 수 있는 라이브러리
- https://www.selenium.dev/documentation/webdriver/browsers 웹 페이지에서 지원하는 브라우저 확인
    - OS : Microsoft Windows, Apple OS X, Linux
    - Browser : Firefox, Edge, Internet Explorer, Safari, Chrome
    - Language : C#, Haskell, Java, Javascript, Object-C, Perl, PHP, Python, R, Ruby

In [48]:
# Jupyter Notebook 설정 : 모든 결과 출력
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [ ]:
!pip install selenium

In [ ]:
!pip install webdriver_manager

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

## 사용 방법

### Web Driver 객체 만들기 및 페이지 이동

In [22]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install())) # 웹드라이버 객체 생성
driver.get("https://www.naver.com/") # 페이지 이동
driver.set_window_size(1024, 768) # 윈도우 크기 설정
driver.execute_script("window.scrollTo(200, 300);") # 브라우져의 스크롤 위치 이동

### Alert 다루기

In [23]:
try:
    alert = driver.switch_to.alert
    print(alert.text)
except:
    print('[NG] alert 없음')

[NG] alert 없음


In [29]:
driver.execute_script("alert('selenium test');") # alert 생성
try:
    alert = driver.switch_to.alert
    print(alert.text) # selenium test
except:
    print('[NG] alert 없음')

selenium test


In [30]:
alert.accept() # '확안' 버튼 클릭

### 버튼 클릭하기

In [31]:
from selenium.webdriver.common.by import By
driver.find_element(By.CSS_SELECTOR,".MyView-module__link_login___HpHMW").click()

### 브라우저 종료하기

In [32]:
driver.quit()

## 네이버 기사 크롤링

In [33]:
article_list = []
def get_article(page):
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    driver.get("https://news.naver.com/section/10" + str(page))
    articles = driver.find_elements(By.CSS_SELECTOR,'#newsct li')
    for article in articles:
        try:
            tmp_elements = article.find_elements(By.CSS_SELECTOR, '.sa_text strong')
            if tmp_elements:
                title = tmp_elements[0].text
            else:
                tmp_elements2 = article.find_elements(By.CSS_SELECTOR, '.ss_text a')
                if tmp_elements2:
                    title = tmp_elements2[0].text
                else:
                    title = "해당 정보 없음"
            article_list.append(title)
        except:
            print("에러 발생!")
    print("end :", page)
    driver.quit()

In [34]:
%%time # Jupyter Notebook에서 해당 셀의 실행 시간을 측정하는 매직 커맨드
for page in range(1, 5):
    get_article(page)

end : 1
end : 2
end : 3
end : 4
CPU times: total: 438 ms
Wall time: 23.4 s


In [36]:
len(article_list), article_list[:10]

(199,
 ['울진군, WHE 2025에서 국가산업단지 비전 제시',
  '정의선 회장 “기아는 정제 안 된 다이아몬드… 계속 도전할 것”',
  'GS건설, LH와 맞손…성북1구역 공공재개발 입찰 완료',
  '10월 경상수지 반토막…해외 주식은 역대 최대',
  '양자전환 산업화 전략 논의…대전서 QX 네트워킹 포럼',
  '',
  '',
  '',
  '',
  ''])

## 네이버 기사 크롤링 (pandas 사용)

In [56]:
#import threading
import pandas as pd
df = pd.DataFrame(columns=["category","title"])

def get_article(driver, page):
    #driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    driver.get("https://news.naver.com/section/10" + str(page))
    category = driver.find_element(By.CSS_SELECTOR, '.ct_snb_h2_a').get_attribute("innerText")
    articles = driver.find_elements(By.CSS_SELECTOR,'#newsct li')
    for article in articles:
        try:
            tmp_elements = article.find_elements(By.CSS_SELECTOR, '.sa_text strong')
            if tmp_elements:
                title = tmp_elements[0].text
            else:
                tmp_elements2 = article.find_elements(By.CSS_SELECTOR, '.ss_text a')
                if tmp_elements2:
                    title = tmp_elements2[0].text
                else:
                    title = "해당 정보 없음"
        except:
            print("에러 발생!")
        df.loc[len(df)] = {"category": category, "title": title}
    print("end :", page+1)
    #driver.quit()

In [57]:
#%%time # Jupyter Notebook에서 해당 셀의 실행 시간을 측정하는 매직 커맨드
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
for page in range(0, 6):
    get_article(driver, page)
driver.quit()

end : 1
end : 2
end : 3
end : 4
end : 5
end : 6


In [63]:
df['category'].value_counts()
df

category
경제       52
IT/과학    52
정치       50
세계       50
사회       49
생활/문화    48
Name: count, dtype: int64

,category,title
0,정치,"정청래 “조희대, 이 대통령 면전서 뻔뻔…연내 사법개혁 완수”"
1,정치,"국민의힘 ""내란재판부·법왜곡죄는 위헌""… 위헌법률심판 제청 예고"
2,정치,"이 대통령, ‘청탁문자 논란’ 김남국 사직서 수리"
3,정치,"박지원 “김남국 사퇴, 칭찬받아 마땅···‘형·누나’는 동료 부르는 민주당 언어 풍토”"
4,정치,서울 ‘제설’ 제대로 됐나 분통 속…추미애도 “여의도서 하남 5시간 걸려”
...,...,...
296,IT/과학,"""공공 클라우드, '권고'에서 '의무'돼야""…국회·산업계, 국가 인프라 재설계 한목소리"
297,IT/과학,건국대 2026 편입 경쟁률 25.28대1
298,IT/과학,"젊은 여성과 나이 많은 사람의 결혼, 이상할 것 없는 이유 [달콤한 사이언스]"
299,IT/과학,"""한국 AI인재 임금 프리미엄, 주요국 꼴찌…미국의 23% 그쳐"""


# END